In [ ]:
import os, re, struct, numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/input/20250907-150827_Rtk.fmnav"  # adjust as needed
MAX_RECORDS = 50000

if not os.path.exists(file_path):
    raise FileNotFoundError(f"❌ File not found: {file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(file_path, "rb") as f:
    data = f.read()

print(f"📦 Read {len(data):,} bytes from file start")

# =====================================================
# 1️⃣  ASCII View  — reveal readable header strings
# =====================================================
ascii_text = ''.join(chr(b) if 32 <= b < 127 else '.' for b in data)

# =====================================================
# 2️⃣ Display preview in chunks (avoid huge single print)
# =====================================================
chunk_size = 512
total_chunks = len(ascii_text) // chunk_size + 1

print(f"\n🔹 Displaying ASCII content in {total_chunks} chunks of {chunk_size} characters:\n")

for i in range(0, len(ascii_text), chunk_size):
    segment = ascii_text[i:i + chunk_size]
    print(f"[{i:06d}-{i+chunk_size:06d}]  {segment}")

print("\n✅ Completed ASCII rendering of entire file.")

📦 Read 1,321,557 bytes from file start

🔹 Displaying ASCII content in 2582 chunks of 512 characters:

[000000-000512]  feimarobotics-slam-rtk............................feima-nav-rtk.......................................................................................FM-982............................................Gu..........R.........4@.........................................................................................................................................11826............................HRPT00-S10C-P.........................................................................................................
[000512-001024]  ...........2310415000012-LR23A6232317189.....................................ff27da9273992dfb.................20X200GO2505150..................................................................................................................................................................................................................................

In [ ]:
patterns = {
    b"$GNGGA": "GNGGA",
    b"$GNRMC": "GNRMC"
}

results = {}

for p_bytes, name in patterns.items():
    # Find all occurrences in raw byte stream
    matches = [m.start() for m in re.finditer(re.escape(p_bytes), data)]
    results[name] = matches


for name, positions in results.items():
    print(f"🔹 {name}: found {len(positions)} occurrences")
    # for pos in positions:
    #     print(f"   → starts at byte offset {pos}")
    print()

print("✅ Completed GNSS sentence search.")

🔹 GNGGA: found 361 occurrences

🔹 GNRMC: found 361 occurrences

✅ Completed GNSS sentence search.


In [ ]:
import re

# Find all GNGGA occurrences
gngga_positions = results["GNGGA"]
print(f"📍 Found {len(gngga_positions)} GNGGA sentences\n")

def extract_nmea_line(start_index):
    """Extract one NMEA line until CR/LF."""
    end = start_index
    while end < len(data) and data[end] not in (10, 13):  # LF or CR
        end += 1
    return data[start_index:end].decode(errors="ignore").strip()

all_gnga_raw = []
for pos in gngga_positions:
    sentence = extract_nmea_line(pos)
    all_gnga_raw.append(sentence)

print("🔹 First 5 raw GNGGA sentences:")
for s in all_gnga_raw[:5]:
    print(s)


📍 Found 361 GNGGA sentences

🔹 First 5 raw GNGGA sentences:
$GNGGA,,,,,,0,,,,,,,,*78
$GNGGA,,,,,,0,,,,,,,,*78
$GNGGA,,,,,,0,,,,,,,,*78
$GNGGA,,,,,,0,,,,,,,,*78
$GNGGA,,,,,,0,,,,,,,,*78


In [ ]:
def safe(parts, idx):
    return parts[idx] if idx < len(parts) else None

def decode_gngga(sentence):
    parts = sentence.split(",")
    decoded = {
        "raw": sentence,
        "type": safe(parts, 0),
        "time_utc": safe(parts, 1),
        "lat_raw": safe(parts, 2),
        "lat_dir": safe(parts, 3),
        "lon_raw": safe(parts, 4),
        "lon_dir": safe(parts, 5),
        "fix_quality": safe(parts, 6),
        "num_sats": safe(parts, 7),
        "hdop": safe(parts, 8),
        "altitude": safe(parts, 9),
        "alt_unit": safe(parts, 10),
    }
    return decoded

decoded_rows = [decode_gngga(s) for s in all_gnga_raw]

df_gga = pd.DataFrame(decoded_rows)
df_gga


,raw,type,time_utc,lat_raw,lat_dir,lon_raw,lon_dir,fix_quality,num_sats,hdop,altitude,alt_unit
0,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
1,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
2,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
3,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
4,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
356,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
357,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
358,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,
359,"$GNGGA,,,,,,0,,,,,,,,*78",$GNGGA,,,,,,0,,,,


In [ ]:
import re

# The correct GNSS binary block header signature
GNSS_SIGNATURE = b"\xAA\x44\x12\x1C"   # from your dump

# Find every occurrence of this packet header in file
hits = [m.start() for m in re.finditer(re.escape(GNSS_SIGNATURE), data)]

print(f"📌 Found {len(hits)} GNSS binary blocks")
print(hits[:20])  # preview


📌 Found 14460 GNSS binary blocks
[1024, 1128, 1204, 1308, 1384, 1488, 1564, 1668, 1744, 1848, 1924, 2028, 2104, 2208, 2284, 2388, 2464, 2568, 2644, 2748]


In [ ]:
OFFSET = 12
FMT = "<Qiii fff H"
SIZE = struct.calcsize(FMT)

rows = []
for h in hits:
    block = data[h + OFFSET : h + OFFSET + SIZE]
    if len(block) != SIZE:
        continue
    try:
        rows.append(struct.unpack(FMT, block))
    except:
        pass

df = pd.DataFrame(rows, columns=[
    "timestamp", "lat_i", "lon_i", "alt_i",
    "v1", "v2", "v3", "quality"
])

print(f"✅ Successfully decoded {len(df)} GNSS rows")
df.head(10)


✅ Successfully decoded 14460 GNSS rows


,timestamp,lat_i,lon_i,alt_i,v1,v2,v3,quality
0,228492260264288,55065,1179649,1,0.0,0.0,0.0,0
1,228492260264288,55066,1179648,1,0.0,0.0,0.0,0
2,228707008629088,55115,1179649,1,0.0,0.0,0.0,0
3,228707008629088,55116,1179649,1,0.0,0.0,0.0,0
4,228921756993888,55165,1179650,1,0.0,0.0,0.0,0
5,228921756993888,55166,1179649,1,0.0,0.0,0.0,0
6,229136505358688,55215,1179650,1,0.0,0.0,0.0,0
7,229136505358688,55216,1179649,1,0.0,0.0,0.0,0
8,229351253723488,55265,1179650,1,0.0,0.0,0.0,0
9,229351253723488,55266,1179649,1,0.0,0.0,0.0,0


In [ ]:
import struct
import pandas as pd

# -------------------------------------------------------
# hits = list of starting byte indexes where GNSS blocks occur
# payload = full binary data read earlier
# -------------------------------------------------------

OFFSET = 12                      # discovered earlier
FMT = "<Qiii fff H"             # timestamp, lat_i, lon_i, alt_i, v1, v2, v3, quality
SIZE = struct.calcsize(FMT)

rows = []

for h in hits:
    block = payload[h + OFFSET : h + OFFSET + SIZE]
    if len(block) != SIZE:
        continue
    try:
        rows.append(struct.unpack(FMT, block))
    except:
        pass

df = pd.DataFrame(rows, columns=[
    "timestamp", "lat_i", "lon_i", "alt_i",
    "v1", "v2", "v3", "quality"
])

print(f"✅ Decoded {len(df)} GNSS rows")
df.head(10)


NameError: name 'payload' is not defined

In [ ]:
# Convert raw fixed-point values
df["lat_deg"] = df["lat_i"] / 1e7
df["lon_deg"] = df["lon_i"] / 1e7
df["alt_m"]   = df["alt_i"] * 1.0

print("📍 First 10 converted GNSS points (unscaled):")
df[["lat_deg", "lon_deg", "alt_m"]].head(10)


In [ ]:
df_valid = df[
    df["lat_deg"].between(-90, 90) &
    df["lon_deg"].between(-180, 180)
]

print(f"📍 Valid coordinate points: {len(df_valid)}")

df_valid.head(10)


In [ ]:
import folium

if df_valid.empty:
    raise ValueError("❌ No valid GNSS coordinates found.")

center_lat = float(df_valid["lat_deg"].mean())
center_lon = float(df_valid["lon_deg"].mean())

print(f"🗺️ Map Center: {center_lat:.6f}, {center_lon:.6f}")

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# plot each GNSS sample
for _, r in df_valid.iterrows():
    folium.CircleMarker(
        location=[r["lat_deg"], r["lon_deg"]],
        radius=2,
        color="blue",
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

# Add center marker
folium.Marker(
    [center_lat, center_lon],
    icon=folium.Icon(color="red"),
    popup="Center"
).add_to(m)

m
